In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F

In [ ]:
data=pd.read_pickle("./dataframe/df_20210912_133401_ONLY_RISKY_EVENTS-6-withFE.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

In [ ]:
#data=data.head(100)

In [ ]:
data.shape

In [ ]:
# ############################### FEATURE ENGINEERING ##########################################
# # Gradient: Miss distance two last CDM
# data["_GRADIENT_MISS_DISTANCE_34"]=(-data.MISS_DISTANCE_3+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# # Gradient: Miss distance first and last CDM
# data["_GRADIENT_MISS_DISTANCE_14"]=(-data.MISS_DISTANCE+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca)
# #Gradient: COLLISSION PROBABILITY two last CDM
# data["_GRADIENT_PC_34"]=(-data.COLLISSION_PROBABILITY_3+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# #Gradient: COLLISSION PROBABILITY first and last CDM
# data["_GRADIENT_PC_14"]=(-data.COLLISSION_PROBABILITY+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca)

In [ ]:
data.shape

In [ ]:
#print(list(data.columns))

In [ ]:
# aux=data
# data=data.loc[:, data.columns.str.endswith('4','TARGET')]
# data["COLLISSION_PROBABILITY_TARGET"]=aux.COLLISSION_PROBABILITY_TARGET
# print(list(data.columns))
# data.shape

In [ ]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [ ]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

In [ ]:
# Y_train = train["COLLISSION_PROBABILITY_TARGET"]
# X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
# Y_test = test["COLLISSION_PROBABILITY_TARGET"]
# X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [ ]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [ ]:
X = X_train
y = Y_train

In [ ]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2_score(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_root_squared_error(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mean_absolute_error(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2_score(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True
            
      # Select metric
      metric='lgb_r2_score'
      metric_feval=lgb_r2_score

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (60, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 35)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      filename="./opt_parameters_bo/opt_parameters_df_singles_{}_{}_RISKY_EVENTS-6_FE.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"),metric)
      a_file = open(filename, "wb")
      pickle.dump(optimizer.max['params'], a_file)
      a_file.close()

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300)

In [ ]:
# improving function
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100,evalm="lgb_r2"):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_rmse(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mae(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True


      metrics_dict= {   "lgb_r2" : lgb_r2,
                        "lgb_rmse":lgb_rmse,
                        "lgb_mae":lgb_mae,
                        "lgb_adjusted_r2": lgb_adjusted_r2
                        }
      # Select metric
      metric=str(evalm)
      metric_feval=metrics_dict.get(str(evalm))

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (60, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 35)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      filename="./opt_parameters_bo/opt_parameters_df_singles_{}_{}_RISKY_EVENTS-6_FE.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"),metric)
      a_file = open(filename, "wb")
      pickle.dump(optimizer.max['params'], a_file)
      a_file.close()

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300,evalm="lgb_r2")

In [ ]:
metrics_dict.get("lgb_r2_score")

In [ ]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

In [ ]:
# filename="opt_parameters_balanced_df_adjusted_{}_corr_mae_RISKY_EVENTS-8.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
# a_file = open(filename, "wb")

# pickle.dump(opt_parameters, a_file)

# a_file.close()

In [ ]:
# filename="./opt_parameters_bo/opt_parameters_balanced_df_adjusted_20210911_215733_corr_lgb_r2_score_RISKY_EVENTS-8-lastcdm-BEST-MODEL.pkl"
# a_file = open(filename,"rb")
# output = pickle.load(a_file)
# opt_parameters=output
# output

In [ ]:
#optimizer.max['params']

In [ ]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'regression_l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 300,
    "n_estimators": 500,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [ ]:
#Construct a gradient boosting model.
#gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
lgbm_train = lgb.Dataset(X, label=y)
#lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [ ]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                #valid_sets=lgbm_eval,
                #verbose_eval=20,
                #eval_metric='lgb_r2_score',
                #early_stopping_rounds=100
                )

In [ ]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


In [ ]:
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [ ]:
# print('The r2 of prediction is:', r2_score(y, Y_pred))
# print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
# print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

In [ ]:
print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

In [ ]:
# df_train = lgb.Dataset(data=X_test, label=Y_test)
# def lgb_adjusted_r2_score(preds, dtrain):
#     labels = dtrain.get_label()
#     n=dtrain.num_data()
#     k=dtrain.num_feature()
#     return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True

In [ ]:
#lgb_adjusted_r2_score(Y_pred, df_train)

### Validating model

In [ ]:
aux_y=pd.DataFrame(Y_test)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result.columns=["y_true","y_predicted"]
result["y_true_10"]=10**result.y_true
result["y_predicted_10"]=10**result.y_predicted
result

In [ ]:
result[result["y_true_10"]>0.00001]

In [ ]:
result[result["y_true_10"]>0.0001]